<a href="https://colab.research.google.com/github/mikhpo/ds-for-non-ds/blob/1-%D0%B4%D0%BE%D0%BC%D0%B0%D1%88%D0%BD%D0%B5%D0%B5-%D0%B7%D0%B0%D0%B4%D0%B0%D0%BD%D0%B8%D0%B5-%D0%BA-23-%D0%BC%D0%B0%D1%8F/notebooks/credit_scoring.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Решение задачи кредитного скоринга

Группа 1:
*   Поляков Михаил Юрьевич
*   Томилин Виктор Александрович
*   Макаров Никита Игоревич

[Репозиторий проекта](https://github.com/mikhpo/ds-for-non-ds) в GitHub.





## Импорт библиотек и настройка среды

Данные хранятся в базе данных PostgreSQL в Yandex Cloud. Для подключения к PostgreSQL-кластеру в Yandex Cloud необходимо загрузить SSL-сертификат.

In [1]:
!mkdir -p ~/.postgresql && \
wget "https://storage.yandexcloud.net/cloud-certs/CA.pem" \
    --output-document ~/.postgresql/root.crt && \
chmod 0600 ~/.postgresql/root.crt

--2023-05-20 10:37:20--  https://storage.yandexcloud.net/cloud-certs/CA.pem
Resolving storage.yandexcloud.net (storage.yandexcloud.net)... 213.180.193.243, 2a02:6b8::1d9
Connecting to storage.yandexcloud.net (storage.yandexcloud.net)|213.180.193.243|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3579 (3.5K) [application/x-x509-ca-cert]
Saving to: ‘/root/.postgresql/root.crt’

/root/.postgresql/r 100%[===================>]   3.50K  --.-KB/s    in 0s      

2023-05-20 10:37:21 (849 MB/s) - ‘/root/.postgresql/root.crt’ saved [3579/3579]



Все библиотеки импортированы в данном блоке в следующем порядке:
1. Модули стандартной библиотеки
2. Сторонние модули из каталога PyPI

In [2]:
import warnings

import psycopg2
import pandas as pd
from IPython.display import display

In [3]:
# Настройки предупреждений.
warnings.filterwarnings("ignore")

Получение данных из таблицы `credit_scoring` в базе данных `db_clients` из PostgreSQL-кластера в Yandex Cloud.

In [4]:
# Создание подключения к базе данных.
conn = psycopg2.connect("""
    host=rc1b-70q90cunwkzx49mj.mdb.yandexcloud.net
    port=6432
    sslmode=verify-full
    dbname=db_clients
    user=mlhs_student
    password=mlhs_student
    target_session_attrs=read-write
""")

In [5]:
# Чтение данных из таблицы SQL-запросом и создание датафрейма Pandas.
query = "SELECT * from public.credit_scoring"

data = pd.read_sql_query(query, conn)

conn.close()

## Разведочный анализ и подготовка данных

Описание данных:
* SeriousDlqin2yrs: Клиент имел просрочку 90 и более дней - целевая переменная.
* RevolvingUtilizationOfUnsecuredLines: Общий баланс средств (total balance on credit cards and personal lines of credit except real estate and no installment debt like car loans divided by the sum of credit limits).
* age: Возраст заемщика
* NumberOfTime30-59DaysPastDueNotWorse: Сколько раз за последние 2 года наблюдалась просрочка 30-59 дней.
* DebtRatio: Ежемесячные расходы (платеж по долгам, алиментам, расходы на проживания) деленные на месячный доход.
* MonthlyIncome: Ежемесячный доход.
* NumberOfOpenCreditLinesAndLoans: Количество открытых кредитов (напрмер, автокредит или ипотека) и кредитных карт.
* NumberOfTimes90DaysLate: Сколько раз наблюдалась просрочка (90 и более дней).
* NumberRealEstateLoansOrLines: Количество кредиов (в том числе под залог жилья)
* RealEstateLoansOrLines: Закодированное количество кредиов (в том числе под залог жилья) - чем больше код буквы, тем больше кредитов
* NumberOfTime60-89DaysPastDueNotWorse: Сколько раз за последние 2 года заемщик задержал платеж на 60-89 дней.
* NumberOfDependents: Количество иждивенцев на попечении (супруги, дети и др).
* GroupAge: закодированная возрастная группа - чем больше код, тем больше возраст.

In [9]:
display(data)

,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents,RealEstateLoansOrLines,GroupAge
0,0,0.221813,43.0,0,0.527888,3280.0,7,0,0,2.0,A,c
1,0,0.602794,25.0,0,0.065868,333.0,2,0,0,0.0,A,b
2,1,0.025656,38.0,0,0.475841,3000.0,7,0,0,2.0,A,c
3,0,0.075427,32.0,0,0.085512,7916.0,6,0,0,0.0,A,b
4,0,0.046560,58.0,0,0.241622,2416.0,9,0,0,0.0,A,d
...,...,...,...,...,...,...,...,...,...,...,...,...
149995,0,0.977899,63.0,0,517.000000,NaN,5,0,1,0.0,A,d
149996,0,0.025449,58.0,0,0.253855,15500.0,7,0,0,2.0,A,d
149997,0,0.058001,83.0,0,0.013997,5000.0,6,0,0,0.0,A,e
149998,0,0.071273,42.0,0,0.008638,6945.0,3,0,0,1.0,A,c
